# Selección de modelos: eliminación hacia atrás

El método de eliminación hacia atrás (backward elimination) es una técnica de selección de variables en modelos de regresión. Comienza con un modelo que incluye todas las variables independientes y, de manera iterativa, elimina la variable menos significativa (usualmente la que tiene el mayor valor de p superior a un umbral, como 0.05). Este proceso se repite hasta que todas las variables restantes son estadísticamente significativas. Es útil para simplificar modelos y mejorar la interpretabilidad sin sacrificar demasiado la capacidad predictiva.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

## Preparación de los datos

In [ ]:
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/data/burritos.xlsx')
df.columns

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
df.dropna(inplace=True)
df.info()

## Análisis de regresión múltiple (Con StatsModels)

In [ ]:
var_ind = ['Cost', 'Hunger', 'Tortilla', 'Temp', 'Meat',
           'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 
           'Synergy', 'Wrap']
X = df[var_ind]
X = sm.add_constant(X)
y = df['overall']

In [ ]:
results = sm.OLS(y, X).fit()
results.summary()

## Eliminación hacia atrás

In [ ]:
def backward_elimination(X, y, significance_level=0.05):
    while True:
        model = sm.OLS(y, X).fit()
        pvalues = model.pvalues.drop('const')  # Excluir la constante
        if pvalues.max() > significance_level:
            excl_var = pvalues.idxmax()
            print(f"Eliminando {excl_var} con p-valor {pvalues[excl_var]:.4f}")
            X = X.drop(columns=[excl_var])
        else:
            break
    return model

In [ ]:
model = backward_elimination(X, y)

In [ ]:
model.summary()

## Eliminación hacia atrás (con librería stepwise_regression)

Para utilizar las funciones de la regresión por pasos se requiere hacer la siguiente instalación:
`!pip install stepwise-regression`

In [ ]:
#!pip install stepwise-regression

In [ ]:
from stepwise_regression import step_reg

In [ ]:
# Modelo completo con todos los predictores
var_ind = ['Cost', 'Hunger', 'Tortilla', 'Temp', 'Meat',
           'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 
           'Synergy', 'Wrap']
X = df[var_ind]
X = sm.add_constant(X)
y = df['overall']

El paquete *Stepwise-regression* tiene dos funciones: *backward-regression* y *forward_regression*. Hay cuatro parámetros:
- X: variables independientes
- y: variable dependiente
- threshold_in: el nivel de significancia para el p-valor
- verbose=False (con 'True' detalla el procedimiento escalonado)

In [ ]:
backselect = step_reg.backward_regression(X, y, 0.05,verbose=True)
backselect

Una vez que identificamos las mejores variables para el modelo, realizamos el análisis